In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score



In [3]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = np.stack([x_train]*3, axis=-1)
x_test = np.stack([x_test]*3, axis=-1)


x_train = np.array([np.pad(img, ((10, 10), (10, 10), (0, 0)), mode='constant') for img in x_train])
x_test = np.array([np.pad(img, ((10, 10), (10, 10), (0, 0)), mode='constant') for img in x_test])


x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)


In [5]:
# Load the pre-trained VGG16 model
input_layer = layers.Input(shape=(48, 48, 3))
model_vgg16 = VGG16(weights='imagenet', input_tensor=input_layer, include_top=False)

for layer in model_vgg16.layers:
    layer.trainable = False
    
last_layer = model_vgg16.output

flatten = layers.Flatten()(last_layer)

dense1 = layers.Dense(64, activation='relu')(flatten)
dense2 = layers.Dense(32, activation='relu')(dense1)

output_layer = layers.Dense(10, activation='softmax')(dense2)

model = models.Model(inputs=input_layer, outputs=output_layer)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 48, 48, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,749,930 (56.27 MB)

 Trainable params: 35,242 (137.66 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
# Extract features using the VGG16 model
feature_extractor = Model(inputs=model_vgg16.input, outputs=flatten)


In [11]:
features_train = feature_extractor.predict(x_train)
features_test = feature_extractor.predict(x_test)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 135s 72ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 68ms/step


2024-06-24 15:19:09.656720: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


In [13]:
# Train a single dense layer (ANN model) on the extracted features
ann_input = layers.Input(shape=(features_train.shape[1],))

dense_layer = layers.Dense(64, activation='relu')(ann_input)

output_layer = layers.Dense(10, activation='softmax')(dense_layer)

ann_model = models.Model(inputs=ann_input, outputs=output_layer)
ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
ann_model.fit(features_train, y_train, epochs=10, batch_size=32, validation_data=(features_test, y_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.2547 - loss: 2.1704 - val_accuracy: 0.3920 - val_loss: 1.5578
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.4326 - loss: 1.5300 - val_accuracy: 0.4985 - val_loss: 1.3726
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.5119 - loss: 1.3661 - val_accuracy: 0.5595 - val_loss: 1.2571
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.5560 - loss: 1.2618 - val_accuracy: 0.5656 - val_loss: 1.1845
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.5955 - loss: 1.1712 - val_accuracy: 0.6522 - val_loss: 1.0617
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.6254 - loss: 1.0913 - val_accuracy: 0.6443 - val_loss: 1.0103
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.6552 - loss: 1.0300 - val_accuracy: 0.7063 - val_loss: 0.9222
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.6759 - loss: 0

In [21]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)  # Reduce to 50 dimensions
features_train_pca = pca.fit_transform(features_train)
features_test_pca = pca.transform(features_test)

In [23]:
# Function to train and evaluate a classifier multiple times and return the accuracies
def evaluate_classifier(classifier, features_train, y_train, features_test, y_test, n_iterations=10):
    accuracies = []
    for _ in range(n_iterations):
        classifier.fit(features_train, y_train)
        y_pred = classifier.predict(features_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    return accuracies

In [25]:
# Evaluate Decision Tree classifier
dt_classifier = DecisionTreeClassifier()
dt_accuracies = evaluate_classifier(dt_classifier, features_train_pca, y_train, features_test_pca, y_test)
dt_mean_accuracy = np.mean(dt_accuracies)
dt_std_accuracy = np.std(dt_accuracies)
print(f"Decision Tree - Mean Accuracy: {dt_mean_accuracy * 100:.2f}%, Std Dev: {dt_std_accuracy * 100:.2f}%")


Decision Tree - Mean Accuracy: 74.82%, Std Dev: 0.20%


In [14]:
accuracy = accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 75.14%
